In [66]:
import numpy as np
import numpy.random as np_rand
import matplotlib.pyplot as plt
import math

# find all empty positions

def find_empty(sud_m):
    empty_l = []
    for x in range(0, len(sud_m)):
        for y in range(0, len(sud_m[x])):
            if sud_m[x, y] == 0:
                empty_l.append((x,y))
                sud_m[x, y] = np_rand.randint(1,10)
    return empty_l

def find_int_empty(matrix):
    empty_l = []
    n = len(matrix)
    for y in range(1,8,3): 
        for x in range(1,8,3):
            block_empty_l = []
            block_dict = {}
            neibs = get_close_neighbours(x, y, n)
            n_neibs = neibs + [(x,y)]
            for i, j in n_neibs: 
                if matrix[i,j] == 0:
                    block_empty_l.append((i,j)) # memorize empty places for each block
                else:
                    block_dict.update({matrix[i,j]:True}) # memorize appeared digits
            empty_l.append(block_empty_l)
            k = 1
            for i, j in block_empty_l: # fill empty spaces with missed digits 
                while block_dict.get(k) != None: # find next digit to fill empty space
                    k += 1
                matrix[i,j] = k
                block_dict.update({k:True})
    return empty_l
                    

# energy

def get_close_neighbours(x, y, n):
    l = []
    for x2 in range(x-1, x+2):
        for y2 in range(y-1, y+2):
            if (-1 < x < n 
                and -1 < y < n 
                and (x != x2 or y != y2) 
                and (0 <= x2 < n) 
                and (0 <= y2 < n)):
                l.append((x2, y2))
    return l

def update_dict(d, key, nval):
    val = d.get(key)
    if val != None:
        val.append(nval)
        d.update({key: val})
    else:
        d.update({key: [nval]})

def energy(matrix):
    n = len(matrix)
    sum = 0    
    for x in range(0,n): # check rows
        row_dict = {}
        for y in range(0,n):
            update_dict(row_dict, matrix[x,y], y)
        rep_l = list(filter(lambda e: e > 1, list(map(lambda l: len(l), row_dict.values()))))
        for e in rep_l:
            sum += e
    for y in range(0,n): # check columns
        col_dict = {}
        for x in range(0,n):
            update_dict(col_dict, matrix[x,y], x)
        rep_l = list(filter(lambda e: e > 1, list(map(lambda l: len(l), col_dict.values()))))
        for e in rep_l:
            sum += e
    for y in range(1,8,3): # check blocks
        for x in range(1,8,3):
            neibs = get_close_neighbours(x, y, n)
            block_dict = {}
            update_dict(block_dict, matrix[x,y],5)
            for i in range(0,len(neibs)):
                k,m = neibs[i]
                update_dict(block_dict, matrix[k,m],i)
            rep_l = list(filter(lambda e: e > 1, list(map(lambda l: len(l), block_dict.values()))))
            for e in rep_l:
                sum += e
    return sum      

# schedule functions

def lin_schedule(t_0, k, n):
    a = np.linspace(0,t_0,n)
    return -a[k]+t_0

def exp_schedule(t_0, k, n):
    return t_0*(0.85**k)

def quad_schedule(t_0, k, n):
    return t_0/(1+2*k^2)

# swap functions

def arbit_swap(matrix, empty_l, neib_num):    
    n = len(empty_l)
    m_l = []
    if neib_num > n:
        neib_num = n
    for i in range(0,neib_num):
        new_m = matrix.copy()
        x, y = empty_l[np_rand.randint(0,len(empty_l))]
        old = new_m[x,y]    
        new = np_rand.randint(1,10)
        while new == old:
            new = np_rand.randint(1,10)
        new_m[x,y] = new
        m_l.append(new_m)
    return m_l

def intel_swap(matrix, empty_l, neib_num):
    m_l = []
    for i in range(0,neib_num):
        new_m = matrix.copy()
        block_num = np_rand.randint(0,len(empty_l))
        block = empty_l[block_num]
        a, b = np_rand.randint(0,len(block),(2))
        while a == b:
            a, b = np_rand.randint(0,len(block),(2))
        x1, y1 = block[a]
        x2, y2 = block[b]
        new_m[x1,y1], new_m[x2,y2] = new_m[x2,y2], new_m[x1,y1]
        m_l.append(new_m)
    return m_l
                                  
# probability

def p(e, e_new, t):
    if t < 1:
        t = 1
    return math.exp(-(e_new - e)/t)

# simulated annealing

def sim_an(s, k_max, t_0, t_1, p, schedule, swap, energy, neighbours_num, empty_l):
    for i in range(0, 5):
        for k in range(0, k_max):
            t = schedule(t_0, k, k_max)
            a = swap(s, empty_l, neighbours_num)
            for s_new in a:
                ne = energy(s_new)
                if ne == 0:
                    return s_new
                if p(energy(s), ne, t) >= np_rand.random():
                    s = s_new
            if math.isclose(t_1, t, rel_tol = 0.01):
                break
    return s

def sim_an_extreme(s, k_max, t_0, t_1, p, schedule, swap, energy, neighbours_num, empty_l):
    for i in range(0, 5):
        for k in range(0, k_max):
            t = schedule(t_0, k, k_max)
            a = swap(s, empty_l, neighbours_num)
            for s_new in a:
                ne = energy(s_new)
                if ne == 0:
                    return s_new
                if p(energy(s), ne, t) >= np_rand.random():
                    s = s_new
    while energy(s) != 0:
        a = swap(s, empty_l, neighbours_num)
        for s_new in a:
            ne = energy(s_new)
            if p(energy(s), ne, t) >= np_rand.random():
                s = s_new
    return s


In [67]:
with open('sudoku.txt') as f:
    read_data = f.readlines()
stream = []
for line in read_data:
    l = line.split()
    stream += l
con_stream = list(map(lambda x: 0 if x == 'x' else int(x), stream))
sud_m = np.array(con_stream).reshape(9,9)
empty_l = find_int_empty(sud_m)

In [68]:
print(sud_m)
print(energy(sud_m))

s = sud_m
schedule = quad_schedule
swap = intel_swap
s_new = sim_an_extreme(s, 500, 1000, 1, p, schedule, swap, energy, 15, empty_l)
print(s_new)
print(energy(s_new))

[[6 9 1 1 8 4 1 2 5]
 [2 4 3 9 2 3 6 9 7]
 [8 5 7 5 7 6 8 4 3]
 [1 6 2 4 7 2 2 3 5]
 [3 9 7 8 1 9 4 9 6]
 [4 5 8 5 6 3 7 1 8]
 [7 1 2 6 1 2 2 3 4]
 [3 9 4 3 9 5 1 6 7]
 [5 6 8 7 4 8 9 5 8]]
78
[[6 9 1 3 8 4 5 7 2]
 [5 4 3 9 2 7 6 8 1]
 [8 7 2 1 5 6 9 4 3]
 [1 6 9 4 7 2 8 3 5]
 [3 5 7 8 1 9 4 2 6]
 [2 8 4 5 6 3 7 1 9]
 [7 1 5 6 3 8 2 9 4]
 [4 3 8 2 9 5 1 6 7]
 [9 2 6 7 4 1 3 5 8]]
0


In [39]:
x = 1
y = 1
matrix = sud_m
n = len(matrix)

e = find_int_empty(matrix)
print(matrix)
print(e)

[[6 9 1 1 8 4 1 2 5]
 [2 4 3 9 2 3 6 9 7]
 [8 5 7 5 7 6 8 4 3]
 [1 6 2 4 7 2 2 3 5]
 [3 9 7 8 1 9 4 9 6]
 [4 5 8 5 6 3 7 1 8]
 [7 1 2 6 1 2 2 3 4]
 [3 9 4 3 9 5 1 6 7]
 [5 6 8 7 4 8 9 5 8]]
[[(0, 2), (1, 0), (2, 1), (2, 2)], [(3, 0), (3, 2), (4, 0), (5, 0), (5, 1), (5, 2), (4, 1)], [(6, 2), (7, 0), (7, 2), (8, 0), (8, 1), (8, 2), (7, 1)], [(0, 3), (1, 5), (2, 3), (2, 4)], [(4, 3), (4, 5)], [(6, 4), (6, 5), (7, 3), (8, 5)], [(0, 6), (0, 7), (0, 8), (1, 6), (1, 8), (2, 6), (1, 7)], [(3, 6), (3, 7), (3, 8), (4, 8), (5, 6), (5, 8), (4, 7)], [(6, 6), (6, 7), (7, 8), (8, 6)]]
